In [4]:
# pygame_car_sim_v2_1.py
# Fix pass:
# - Robust anti-overlap: per-lane leader/follower spacing + post-step separation
# - Consistent hazard stop: ROADBLOCK & ACCIDENT stop **all** cars ~10 m before
# - Predictive braking: smooth slowdown when <= 60 m from hazard/leader
# - Safer spawning buffers
#
# Controls unchanged from v2.
# Requires: pygame (pip install pygame)

import math
import random
import pygame as pg
from dataclasses import dataclass
from typing import Optional, List, Tuple

# ----------------------------- Config -----------------------------
WIN_W, WIN_H = 1100, 560
LANE_W = 60
ROAD_W = 2 * LANE_W
PX_PER_M = 1.0
CAM_MARGIN_X = 360

# Colors
ROAD_COLOR = (52, 54, 60)
LANE_LINE = (220, 220, 220)
BG = (24, 26, 30)
TEXT = (240, 240, 240)
MUTED = (170, 170, 170)
ACC_COLOR = (255, 196, 64)
RB_COLOR = (200, 120, 255)
EGO_COLOR = (60, 180, 255)
AMB_COLOR = (255, 90, 90)
NPC_COLOR = (120, 210, 150)

FONT_NAME = "consolas"

CAR_L = 48
CAR_W = 26
GAP_MIN = 26.0           # strict min bumper-to-bumper gap
BRAKE_LOOKAHEAD = 60.0   # start soft braking if within this distance

# ----------------------------- Model -----------------------------
@dataclass
class Car:
    name: str
    x: float = 0.0
    v: float = 15.0
    vmax: float = 20.0
    lane: int = 1
    active: bool = True
    color: Tuple[int,int,int] = (120, 210, 150)
    ai: bool = False

    def step(self, dt: float):
        if not self.active:
            return
        # relax toward vmax
        if self.v < self.vmax:
            self.v = min(self.vmax, self.v + 3.0 * dt)
        elif self.v > self.vmax:
            self.v = max(self.vmax, self.v - 5.0 * dt)
        self.v = max(0.0, self.v)
        self.x += self.v * dt

class World:
    def __init__(self, length=2200.0):
        self.t = 0.0
        self.dt = 1.0 / 60.0
        self.length = length

        self.ego = Car("EGO", x=0.0, v=16.0, vmax=22.0, lane=1, active=True, color=EGO_COLOR)
        self.amb = Car("AMB", x=-160.0, v=0.0, vmax=32.0, lane=1, active=False, color=AMB_COLOR)

        self.traffic: List[Car] = []
        self.spawn_traffic = True
        self._next_spawn_t = 0.0

        self.accident_x: Optional[float] = None
        self.roadblock_x: Optional[float] = None

        self.STOP_BUFFER = 10.0
        self.yielding = False
        self.scheduled_amb: List[float] = []

        self.log: List[str] = []

    # --------- Scenario actions ---------
    def dispatch_ambulance(self):
        if not self.amb.active:
            self.amb.active = True
            self.amb.x = self.ego.x - 160.0
            self.amb.v = 30.0
            self.log.append(f"{self.t:6.2f}s: 🚑 Ambulance dispatched at x={self.amb.x:.1f}")

    def schedule_ambulance(self, after_s: float = 5.0):
        when = self.t + max(0.0, after_s)
        self.scheduled_amb.append(when)
        self.log.append(f"{self.t:6.2f}s: 🚑 Ambulance scheduled for t={when:.2f}s")

    def place_accident(self, x: float):
        self.accident_x = x
        self.log.append(f"{self.t:6.2f}s: ⚠️ Accident placed at x={x:.1f}")

    def clear_accident(self):
        self.accident_x = None
        self.log.append(f"{self.t:6.2f}s: ⚠️ Accident cleared")

    def place_roadblock(self, x: float):
        self.roadblock_x = x
        self.log.append(f"{self.t:6.2f}s: ⛔ Roadblock placed at x={x:.1f}")

    def clear_roadblock(self):
        self.roadblock_x = None
        self.log.append(f"{self.t:6.2f}s: ⛔ Roadblock cleared")

    # --------- Traffic system ---------
    def _spawn_traffic_logic(self):
        if not self.spawn_traffic:
            return
        if self.t < self._next_spawn_t:
            return
        self._next_spawn_t = self.t + random.uniform(1.6, 3.2)

        lane = random.choice([1, 2])
        ahead = random.random() < 0.7
        base_x = self.ego.x + random.uniform(250, 700) if ahead else self.ego.x - random.uniform(220, 420)

        # Avoid spawning too close to existing vehicles
        for c in [self.ego, self.amb] + self.traffic:
            if c.lane == lane and abs(c.x - base_x) < 120:
                return  # too close; skip spawn

        v = random.uniform(12.0, 24.0)
        car = Car("NPC", x=base_x, v=v, vmax=v, lane=lane, active=True, color=NPC_COLOR, ai=True)
        self.traffic.append(car)

        # cull far cars to keep load low
        self.traffic = [c for c in self.traffic if (self.ego.x - 900) <= c.x <= (self.ego.x + 1300)]

    def _find_leader(self, car: Car) -> Optional[Car]:
        leader = None
        best_d = 1e9
        for c in [self.amb] + self.traffic + [self.ego]:
            if c is car or not c.active or c.lane != car.lane:
                continue
            if c.x > car.x:
                d = c.x - car.x
                if d < best_d:
                    best_d = d; leader = c
        return leader

    def _traffic_following(self, car: Car):
        leader = self._find_leader(car)
        if not leader:
            # relax upward a bit when free
            car.vmax = min(26.0, max(car.vmax, car.v + 2.0 * self.dt))
            return
        d = leader.x - car.x
        rel_v = max(0.0, car.v - leader.v)  # how much faster we travel

        # Predictive soft braking if within lookahead
        if d < BRAKE_LOOKAHEAD:
            # target vmax scales with available distance (keep at least GAP_MIN)
            target_v = max(0.0, min(car.vmax, (d - GAP_MIN) * 0.6))
            car.vmax = min(car.vmax, target_v)
            # emergency braking if dangerously close
            if d < GAP_MIN:
                car.v = max(0.0, min(car.v, leader.v - 0.2))
        else:
            # far from leader -> allow gentle speed-up
            car.vmax = min(26.0, car.vmax + 2.0 * self.dt)

    # --------- Hazard logic (applies to ALL cars) ---------
    def _hazard_stop_logic(self, car: Car):
        # returns True if forced stop by hazard
        hx_list = []
        if self.accident_x is not None:
            hx_list.append(self.accident_x)
        if self.roadblock_x is not None:
            hx_list.append(self.roadblock_x)
        forced = False
        for hx in hx_list:
            dist = hx - car.x
            # predictive: slow when within lookahead; hard stop at buffer
            if dist <= BRAKE_LOOKAHEAD and dist > self.STOP_BUFFER:
                # scale target with distance remaining to buffer
                target_v = max(0.0, (dist - self.STOP_BUFFER) * 0.5)
                car.vmax = min(car.vmax, target_v)
            if dist <= self.STOP_BUFFER:
                car.vmax = 0.0
                car.v = 0.0
                forced = True
        return forced

    # --------- Post integration separation (anti-overlap) ---------
    def _separate_per_lane(self):
        for lane in (1, 2):
            cars = [c for c in ([self.amb] + self.traffic + [self.ego]) if c.active and c.lane == lane]
            cars.sort(key=lambda c: c.x)  # rear -> front
            for i in range(len(cars) - 1):
                rear, front = cars[i], cars[i+1]
                if front.x - rear.x < GAP_MIN:
                    # push rear back and match speed
                    rear.x = front.x - GAP_MIN
                    rear.v = min(rear.v, front.v)
                    rear.vmax = min(rear.vmax, front.v)

    def step(self):
        # spawn/manage traffic
        self._spawn_traffic_logic()

        # ambulance + yielding
        self._ambulance_logic()

        # hazards apply to all cars (roadblock blocks both lanes; accident too in this model)
        self._hazard_stop_logic(self.ego)
        if self.amb.active:
            self._hazard_stop_logic(self.amb)
        for c in self.traffic:
            self._hazard_stop_logic(c)

        # Traffic AI
        for c in self.traffic:
            if c.ai:
                self._traffic_following(c)

        # Integrate
        self.ego.step(self.dt)
        if self.amb.active:
            self.amb.step(self.dt)
        for c in self.traffic:
            c.step(self.dt)

        # Separation pass to kill overlaps
        self._separate_per_lane()

        self.t += self.dt

    # --------- Yielding / Ambulance ---------
    def _ambulance_logic(self):
        # trigger schedules
        if self.scheduled_amb:
            still = []
            for when in self.scheduled_amb:
                if self.t >= when and not self.amb.active:
                    self.dispatch_ambulance()
                else:
                    still.append(when)
            self.scheduled_amb = still

        def heard():
            return self.amb.active and (self.amb.x < self.ego.x) and ((self.ego.x - self.amb.x) <= 150.0)

        if heard():
            if not self.yielding:
                self.yielding = True
                self.ego.lane = 2
                self.log.append(f"{self.t:6.2f}s: EGO yields → lane 2")
            self.ego.vmax = max(10.0, self.ego.vmax - 0.9 * self.dt)
        else:
            if self.yielding and (self.amb.x > self.ego.x + 20.0):
                self.yielding = False
                self.ego.lane = 1
                self.ego.vmax = max(self.ego.vmax, 22.0)
                self.log.append(f"{self.t:6.2f}s: EGO resumes → lane 1")

        # ambulance lane swap if boxed with ego
        if self.amb.active:
            if abs(self.amb.x - self.ego.x) < 18.0 and self.amb.lane == self.ego.lane:
                self.amb.lane = 2 if self.amb.lane == 1 else 1
                self.log.append(f"{self.t:6.2f}s: 🚑 overtakes via lane {self.amb.lane}")

# ----------------------------- View / UI -----------------------------
class UI:
    def __init__(self, screen: pg.Surface, font: pg.font.Font):
        self.screen = screen
        self.font = font

    def draw_road(self, camera_x: float):
        cy = WIN_H // 2
        road_rect = pg.Rect(0, cy - ROAD_W//2, WIN_W, ROAD_W)
        pg.draw.rect(self.screen, ROAD_COLOR, road_rect, border_radius=14)
        pg.draw.rect(self.screen, (38,40,45), (0, cy - ROAD_W//2 - 14, WIN_W, 14))
        pg.draw.rect(self.screen, (38,40,45), (0, cy + ROAD_W//2, WIN_W, 14))
        # dashed center line
        mid_y = cy
        dash = 26
        gap = 20
        start = int((camera_x % (dash + gap)) - (dash + gap))
        for x in range(start, WIN_W + dash + gap, dash + gap):
            pg.draw.rect(self.screen, LANE_LINE, (x, mid_y - 2, dash, 4), border_radius=2)

    def lane_y(self, lane: int) -> int:
        cy = WIN_H // 2
        return cy - LANE_W//2 if lane == 1 else cy + LANE_W//2

    def world_to_screen(self, wx: float, camera_x: float) -> float:
        return (wx - camera_x) * PX_PER_M

    def draw_car(self, car: Car, camera_x: float):
        cx = self.world_to_screen(car.x, camera_x)
        cy = self.lane_y(car.lane)
        # shadow
        shadow = pg.Rect(0, 0, CAR_L + 8, CAR_W + 4)
        shadow.center = (cx, cy + 6)
        s = pg.Surface(shadow.size, pg.SRCALPHA)
        pg.draw.ellipse(s, (0,0,0,110), s.get_rect())
        self.screen.blit(s, shadow.topleft)
        # body
        body = pg.Rect(0, 0, CAR_L, CAR_W)
        body.center = (cx, cy)
        pg.draw.rect(self.screen, car.color, body, border_radius=8)
        # windshield
        roof = body.inflate(-CAR_L*0.45, -CAR_W*0.25)
        pg.draw.rect(self.screen, (230, 240, 255), roof, border_radius=6)
        # wheels
        for ox in (-CAR_L*0.35, CAR_L*0.35):
            wheel = pg.Rect(0, 0, 8, CAR_W+2)
            wheel.center = (cx + ox, cy)
            pg.draw.rect(self.screen, (20,20,20), wheel, border_radius=3)
        # ambulance blink
        if car.name == "AMB":
            blink = (int(pg.time.get_ticks()/120) % 2) == 0
            if blink:
                pg.draw.rect(self.screen, (255,240,120), (body.right-6, body.top+4, 6, 6), border_radius=2)
                pg.draw.rect(self.screen, (255,240,120), (body.right-6, body.bottom-10, 6, 6), border_radius=2)
        # stroke
        pg.draw.rect(self.screen, (0,0,0), body, 2, border_radius=8)

    def draw_marker(self, wx: float, camera_x: float, color: Tuple[int,int,int], label: str):
        sx = self.world_to_screen(wx, camera_x)
        cy = WIN_H // 2
        pg.draw.line(self.screen, color, (sx, cy - ROAD_W//2 - 16), (sx, cy + ROAD_W//2 + 16), 4)
        tag = self.font.render(label, True, color)
        self.screen.blit(tag, (sx - tag.get_width()//2, cy + ROAD_W//2 + 20))

    def draw_hud(self, world: World, camera_x: float, mode: str, paused: bool, show_help: bool, spawn_on: bool):
        lines = [
            f"t={world.t:5.1f}s  EGO x={world.ego.x:6.1f}m  v={world.ego.v:4.1f}  lane={world.ego.lane}",
            f"AMB {'ON ' if world.amb.active else 'OFF'} x={world.amb.x:6.1f} v={world.amb.v:4.1f} lane={world.amb.lane}",
            f"mode=[{mode}]  pause={'YES' if paused else 'no '}  vmax={world.ego.vmax:4.1f}  traffic={'ON' if spawn_on else 'OFF'}",
            f"cars={len(world.traffic)+2}",
        ]
        y = 10
        for s in lines:
            surf = self.font.render(s, True, TEXT)
            self.screen.blit(surf, (12, y))
            y += 20
        if world.accident_x is not None:
            sx = self.world_to_screen(world.accident_x, camera_x)
            pg.draw.circle(self.screen, ACC_COLOR, (int(sx), 40), 7)
            self.screen.blit(self.font.render("ACC", True, ACC_COLOR), (int(sx)-14, 52))
        if world.roadblock_x is not None:
            sx = self.world_to_screen(world.roadblock_x, camera_x)
            pg.draw.circle(self.screen, RB_COLOR, (int(sx), 40), 7)
            self.screen.blit(self.font.render("RB", True, RB_COLOR), (int(sx)-10, 52))
        if show_help:
            self.draw_help_overlay()

    def draw_help_overlay(self):
        pad = 10
        msg = [
            "Controls:",
            "  1: Accident mode (click to place)",
            "  2: Roadblock mode (click to place)",
            "  A / Shift+A: Ambulance now / in 5s",
            "  C: Clear hazard near cursor  |  X: Clear ALL",
            "  R: Responders clear scene (unblocks)",
            "  W/S: Ego speed up/down  |  T: Toggle traffic",
            "  SPACE: Pause/Resume  |  H: Help",
            "  ESC: Back to Menu  |  Q: Quit",
        ]
        w = 560
        h = 22 * len(msg) + pad*2
        rect = pg.Rect(WIN_W - w - 20, 20, w, h)
        s = pg.Surface((rect.w, rect.h), pg.SRCALPHA)
        s.fill((0,0,0,170))
        self.screen.blit(s, rect.topleft)
        y = rect.top + pad
        for line in msg:
            surf = self.font.render(line, True, TEXT)
            self.screen.blit(surf, (rect.left + pad, y))
            y += 22

# ----------------------------- Menu -----------------------------
class Menu:
    def __init__(self, screen: pg.Surface, bigfont: pg.font.Font, font: pg.font.Font):
        self.screen = screen
        self.bigfont = bigfont
        self.font = font

    def draw(self):
        self.screen.fill(BG)
        title = self.bigfont.render("Car Simulation", True, TEXT)
        sub = self.font.render("Ambulance · Accident · Roadblock (v2.1)", True, MUTED)
        self.screen.blit(title, (WIN_W//2 - title.get_width()//2, 60))
        self.screen.blit(sub, (WIN_W//2 - sub.get_width()//2, 110))

        btns = [
            ("1) Ambulance preset", 1),
            ("2) Accident preset", 2),
            ("3) Roadblock preset", 3),
            ("Q) Quit", -1),
        ]
        y = 200
        for label, _ in btns:
            surf = self.font.render(label, True, TEXT)
            rect = surf.get_rect(center=(WIN_W//2, y))
            pg.draw.rect(self.screen, (60,60,60), rect.inflate(30,12), border_radius=8)
            self.screen.blit(surf, rect)
            y += 64

        tip = self.font.render("Tip: Hazards now stop ALL cars. Use R to clear.", True, MUTED)
        self.screen.blit(tip, (WIN_W//2 - tip.get_width()//2, WIN_H - 60))

    def interact(self) -> int:
        while True:
            for e in pg.event.get():
                if e.type == pg.QUIT:
                    return -1
                if e.type == pg.KEYDOWN:
                    if e.key == pg.K_1: return 1
                    if e.key == pg.K_2: return 2
                    if e.key == pg.K_3: return 3
                    if e.key in (pg.K_ESCAPE, pg.K_q): return -1
            self.draw()
            pg.display.flip()

# ----------------------------- App -----------------------------
class App:
    def __init__(self):
        pg.init()
        pg.display.set_caption("Car Sim v2.1 — Consistent Stops + Anti-Overlap")
        self.screen = pg.display.set_mode((WIN_W, WIN_H))
        self.font = pg.font.SysFont(FONT_NAME, 16)
        self.bigfont = pg.font.SysFont(FONT_NAME, 42, bold=True)
        self.ui = UI(self.screen, self.font)
        self.clock = pg.time.Clock()

    def run(self):
        while True:
            choice = self.menu()
            if choice == -1:
                break
            self.sim(choice)

    def menu(self) -> int:
        m = Menu(self.screen, self.bigfont, self.font)
        return m.interact()

    def sim(self, choice: int):
        world = World(length=2200.0)
        paused = False
        show_help = True
        mode = "accident"
        camera_x = 0.0

        # presets
        if choice == 1:
            world.schedule_ambulance(8.0)
        elif choice == 2:
            world.place_accident(400.0)
        elif choice == 3:
            world.place_roadblock(900.0)

        running = True
        while running:
            for e in pg.event.get():
                if e.type == pg.QUIT:
                    pg.quit(); raise SystemExit
                if e.type == pg.KEYDOWN:
                    if e.key == pg.K_ESCAPE:
                        running = False
                    elif e.key == pg.K_q:
                        pg.quit(); raise SystemExit
                    elif e.key == pg.K_h:
                        show_help = not show_help
                    elif e.key == pg.K_SPACE:
                        paused = not paused
                    elif e.key == pg.K_1:
                        mode = "accident"
                    elif e.key == pg.K_2:
                        mode = "roadblock"
                    elif e.key == pg.K_a:
                        if pg.key.get_mods() & pg.KMOD_SHIFT:
                            world.schedule_ambulance(5.0)
                        else:
                            world.dispatch_ambulance()
                    elif e.key == pg.K_c:
                        mx, my = pg.mouse.get_pos()
                        wx = camera_x + mx / PX_PER_M
                        nearest = None
                        if world.accident_x is not None:
                            nearest = (abs(world.accident_x - wx), 'acc')
                        if world.roadblock_x is not None:
                            d = abs(world.roadblock_x - wx)
                            if nearest is None or d < nearest[0]:
                                nearest = (d, 'rb')
                        if nearest and nearest[0] < 40:
                            if nearest[1] == 'acc': world.clear_accident()
                            else: world.clear_roadblock()
                    elif e.key == pg.K_x:
                        world.clear_accident(); world.clear_roadblock()
                    elif e.key == pg.K_r:
                        world.clear_accident(); world.clear_roadblock()
                    elif e.key == pg.K_w:
                        world.ego.vmax = min(32.0, world.ego.vmax + 1.0)
                    elif e.key == pg.K_s:
                        world.ego.vmax = max(0.0, world.ego.vmax - 1.0)
                    elif e.key == pg.K_t:
                        world.spawn_traffic = not world.spawn_traffic
                if e.type == pg.MOUSEBUTTONDOWN and e.button == 1:
                    mx, my = e.pos
                    wx = camera_x + mx / PX_PER_M
                    if mode == "accident":
                        world.place_accident(wx)
                    elif mode == "roadblock":
                        world.place_roadblock(wx)
                if e.type == pg.MOUSEBUTTONDOWN and e.button == 3:
                    mx, my = e.pos
                    wx = camera_x + mx / PX_PER_M
                    if world.accident_x is not None and (world.roadblock_x is None or abs(world.accident_x - wx) < abs(world.roadblock_x - wx)):
                        if abs(world.accident_x - wx) < 40:
                            world.clear_accident()
                    elif world.roadblock_x is not None and abs(world.roadblock_x - wx) < 40:
                        world.clear_roadblock()

            if not paused:
                world.step()

            target = world.ego.x - CAM_MARGIN_X
            camera_x += (target - camera_x) * 0.15
            camera_x = max(0.0, camera_x)

            self.screen.fill(BG)
            self.ui.draw_road(camera_x)
            if world.accident_x is not None:
                self.ui.draw_marker(world.accident_x, camera_x, ACC_COLOR, "ACCIDENT")
            if world.roadblock_x is not None:
                self.ui.draw_marker(world.roadblock_x, camera_x, RB_COLOR, "ROADBLOCK")
            if world.amb.active:
                self.ui.draw_car(world.amb, camera_x)
            for c in world.traffic:
                self.ui.draw_car(c, camera_x)
            self.ui.draw_car(world.ego, camera_x)
            self.ui.draw_hud(world, camera_x, mode, paused, show_help, world.spawn_traffic)

            pg.display.flip()
            self.clock.tick(60)

        # back to menu ends sim

if __name__ == "__main__":
    pg.init()
    App().run()


SystemExit: 

In [5]:
# pygame_car_sim_v2_2.py
# Stability pass:
# - Deterministic hazard stop: exact stop-point clamp for ALL cars (no rolling past)
# - Proper release after clearing hazards (cars resume automatically)
# - Stronger anti-overlap solver per-lane after integration
# - Safer spawns and strict lane centering (no "between lanes")
#
# Controls unchanged (1/2 place hazard, A/Shift+A ambulance, C/X/R clear, T traffic, etc.)

import random
import pygame as pg
from dataclasses import dataclass
from typing import Optional, List, Tuple

# ----------------------------- Config -----------------------------
WIN_W, WIN_H = 1100, 560
LANE_W = 60
ROAD_W = 2 * LANE_W
PX_PER_M = 1.0
CAM_MARGIN_X = 360

# Colors
ROAD_COLOR = (52, 54, 60)
LANE_LINE = (220, 220, 220)
BG = (24, 26, 30)
TEXT = (240, 240, 240)
MUTED = (170, 170, 170)
ACC_COLOR = (255, 196, 64)
RB_COLOR = (200, 120, 255)
EGO_COLOR = (60, 180, 255)
AMB_COLOR = (255, 90, 90)
NPC_COLOR = (120, 210, 150)

FONT_NAME = "consolas"

CAR_L = 48
CAR_W = 26
MIN_GAP = 30.0           # enforced bumper gap after step
BRAKE_LOOKAHEAD = 80.0   # predictive braking distance (m)
STOP_BUFFER = 12.0       # stop this far BEFORE hazard line

# ----------------------------- Model -----------------------------
@dataclass
class Car:
    name: str
    x: float = 0.0
    v: float = 15.0
    vmax: float = 20.0
    lane: int = 1
    active: bool = True
    color: Tuple[int,int,int] = (120, 210, 150)
    ai: bool = False
    base_vmax: float = 20.0     # cruise target to return to after stops
    stop_reason: Optional[str] = None  # 'hazard' or 'leader'
    stop_at: Optional[float] = None    # world x where we clamped

    def step(self, dt: float):
        if not self.active:
            return
        # relax toward vmax
        if self.v < self.vmax:
            self.v = min(self.vmax, self.v + 3.0 * dt)
        elif self.v > self.vmax:
            self.v = max(self.vmax, self.v - 5.0 * dt)
        if self.v < 0: self.v = 0.0
        self.x += self.v * dt

class World:
    def __init__(self, length=2200.0):
        self.t = 0.0
        self.dt = 1.0 / 60.0
        self.length = length

        self.ego = Car("EGO", x=0.0, v=16.0, vmax=22.0, base_vmax=22.0, lane=1, active=True, color=EGO_COLOR)
        self.amb = Car("AMB", x=-180.0, v=0.0, vmax=32.0, base_vmax=32.0, lane=1, active=False, color=AMB_COLOR)

        self.traffic: List[Car] = []
        self.spawn_traffic = True
        self._next_spawn_t = 0.0

        self.accident_x: Optional[float] = None
        self.roadblock_x: Optional[float] = None

        self.yielding = False
        self.scheduled_amb: List[float] = []
        self.log: List[str] = []

    # --------- Scenario actions ---------
    def dispatch_ambulance(self):
        if not self.amb.active:
            self.amb.active = True
            self.amb.x = self.ego.x - 180.0
            self.amb.v = 30.0
            self.log.append(f"{self.t:6.2f}s: 🚑 Ambulance dispatched at x={self.amb.x:.1f}")

    def schedule_ambulance(self, after_s: float = 5.0):
        when = self.t + max(0.0, after_s)
        self.scheduled_amb.append(when)
        self.log.append(f"{self.t:6.2f}s: 🚑 Ambulance scheduled for t={when:.2f}s")

    def place_accident(self, x: float):
        self.accident_x = x
        self.log.append(f"{self.t:6.2f}s: ⚠️ Accident placed at x={x:.1f}")

    def clear_accident(self):
        self.accident_x = None
        self.log.append(f"{self.t:6.2f}s: ⚠️ Accident cleared")

    def place_roadblock(self, x: float):
        self.roadblock_x = x
        self.log.append(f"{self.t:6.2f}s: ⛔ Roadblock placed at x={x:.1f}")

    def clear_roadblock(self):
        self.roadblock_x = None
        self.log.append(f"{self.t:6.2f}s: ⛔ Roadblock cleared")

    # --------- Helpers ---------
    def _nearest_hazard_ahead(self, x: float) -> Optional[float]:
        hs = [h for h in [self.accident_x, self.roadblock_x] if h is not None and h >= x - CAR_L]
        return min(hs) if hs else None

    def _find_leader(self, car: Car) -> Optional[Car]:
        leader = None
        best_d = 1e9
        for c in [self.amb] + self.traffic + [self.ego]:
            if c is car or not c.active or c.lane != car.lane:
                continue
            if c.x > car.x:
                d = c.x - car.x
                if d < best_d:
                    best_d = d; leader = c
        return leader

    # --------- Traffic system ---------
    def _spawn_traffic_logic(self):
        if not self.spawn_traffic or self.t < self._next_spawn_t:
            return
        self._next_spawn_t = self.t +  random.uniform(1.8, 3.2)

        lane = 1 if random.random() < 0.5 else 2
        ahead = random.random() < 0.7
        base_x = self.ego.x + random.uniform(300, 750) if ahead else self.ego.x - random.uniform(250, 450)

        # strict spacing to all cars in same lane
        for c in [self.ego, self.amb] + self.traffic:
            if c.lane == lane and abs(c.x - base_x) < (MIN_GAP + 60):
                return

        v = random.uniform(13.0, 24.0)
        car = Car("NPC", x=base_x, v=v, vmax=v, base_vmax=v, lane=lane, active=True, color=NPC_COLOR, ai=True)
        self.traffic.append(car)
        # cull distant
        self.traffic = [c for c in self.traffic if (self.ego.x - 900) <= c.x <= (self.ego.x + 1400)]

    # --------- Control / physics passes ---------
    def _ambulance_logic(self):
        # schedules
        if self.scheduled_amb:
            still = []
            for when in self.scheduled_amb:
                if self.t >= when and not self.amb.active:
                    self.dispatch_ambulance()
                else:
                    still.append(when)
            self.scheduled_amb = still

        heard = self.amb.active and (self.amb.x < self.ego.x) and ((self.ego.x - self.amb.x) <= 150.0)
        if heard:
            if not self.yielding:
                self.yielding = True
                self.ego.lane = 2
                self.log.append(f"{self.t:6.2f}s: EGO yields → lane 2")
            # nudge down the max while yielding
            self.ego.vmax = max(10.0, self.ego.vmax - 0.9 * self.dt)
        else:
            if self.yielding and (self.amb.x > self.ego.x + 20.0):
                self.yielding = False
                self.ego.lane = 1
                self.ego.vmax = max(self.ego.vmax, self.ego.base_vmax)
                self.log.append(f"{self.t:6.2f}s: EGO resumes → lane 1")
        # ambulance side-swap if boxed with ego
        if self.amb.active and abs(self.amb.x - self.ego.x) < 18.0 and self.amb.lane == self.ego.lane:
            self.amb.lane = 2 if self.amb.lane == 1 else 1
            self.log.append(f"{self.t:6.2f}s: 🚑 overtakes via lane {self.amb.lane}")

    def _apply_hazards(self, car: Car):
        hx = self._nearest_hazard_ahead(car.x)
        if hx is None:
            # release if we were stopped for hazard
            if car.stop_reason == 'hazard':
                car.vmax = max(car.vmax, car.base_vmax)
                car.stop_reason = None
                car.stop_at = None
            return
        stop_point = hx - STOP_BUFFER
        # predictive braking: if within lookahead but before stop line
        dist = stop_point - car.x
        if dist <= BRAKE_LOOKAHEAD and dist > 0:
            target = max(0.0, min(car.vmax, dist * 0.6))
            car.vmax = min(car.vmax, target)
        # hard clamp: don't cross stop line this step
        projected = car.x + max(0.0, car.v) * self.dt
        if projected >= stop_point:
            car.x = min(stop_point, car.x)  # clamp to line (or stay behind)
            car.v = 0.0
            car.vmax = 0.0
            car.stop_reason = 'hazard'
            car.stop_at = stop_point

    def _car_following(self, car: Car):
        leader = self._find_leader(car)
        if not leader:
            # free road → relax to base
            if car.vmax < car.base_vmax:
                car.vmax = min(car.base_vmax, car.vmax + 2.0 * self.dt)
            return
        d = leader.x - car.x
        if d < BRAKE_LOOKAHEAD:
            target = max(0.0, (d - MIN_GAP) * 0.6)
            car.vmax = min(car.vmax, target)
            if d < MIN_GAP:
                car.v = min(car.v, leader.v)
        else:
            car.vmax = min(car.base_vmax, car.vmax + 2.0 * self.dt)

    def _separate_per_lane(self):
        for lane in (1, 2):
            cars = [c for c in ([self.amb] + self.traffic + [self.ego]) if c.active and c.lane == lane]
            cars.sort(key=lambda c: c.x)
            for i in range(len(cars) - 1):
                rear, front = cars[i], cars[i+1]
                if rear.x > front.x - MIN_GAP:
                    rear.x = front.x - MIN_GAP
                    if rear.x < 0: rear.x = 0
                    rear.v = min(rear.v, front.v)
                    rear.vmax = min(rear.vmax, front.v)

    def step(self):
        # spawn traffic
        self._spawn_traffic_logic()

        # global controls
        self._ambulance_logic()

        # hazards & following for all cars
        self._apply_hazards(self.ego)
        if self.amb.active:
            self._apply_hazards(self.amb)
        for c in self.traffic:
            self._apply_hazards(c)
        for c in self.traffic:
            if c.ai:
                self._car_following(c)

        # integrate
        self.ego.step(self.dt)
        if self.amb.active:
            self.amb.step(self.dt)
        for c in self.traffic:
            c.step(self.dt)

        # fix overlaps strictly
        self._separate_per_lane()

        self.t += self.dt

# ----------------------------- View / UI -----------------------------
class UI:
    def __init__(self, screen: pg.Surface, font: pg.font.Font):
        self.screen = screen
        self.font = font

    def draw_road(self, camera_x: float):
        cy = WIN_H // 2
        road_rect = pg.Rect(0, cy - ROAD_W//2, WIN_W, ROAD_W)
        pg.draw.rect(self.screen, ROAD_COLOR, road_rect, border_radius=14)
        pg.draw.rect(self.screen, (38,40,45), (0, cy - ROAD_W//2 - 14, WIN_W, 14))
        pg.draw.rect(self.screen, (38,40,45), (0, cy + ROAD_W//2, WIN_W, 14))
        # dashed center line
        mid_y = cy
        dash = 26
        gap = 20
        start = int((camera_x % (dash + gap)) - (dash + gap))
        for x in range(start, WIN_W + dash + gap, dash + gap):
            pg.draw.rect(self.screen, LANE_LINE, (x, mid_y - 2, dash, 4), border_radius=2)

    def lane_y(self, lane: int) -> int:
        cy = WIN_H // 2
        # exact centers: never between lanes
        return cy - LANE_W//2 if lane == 1 else cy + LANE_W//2

    def world_to_screen(self, wx: float, camera_x: float) -> float:
        return (wx - camera_x) * PX_PER_M

    def draw_car(self, car: Car, camera_x: float):
        cx = self.world_to_screen(car.x, camera_x)
        cy = self.lane_y(car.lane)
        # shadow
        shadow = pg.Rect(0, 0, CAR_L + 8, CAR_W + 4)
        shadow.center = (cx, cy + 6)
        s = pg.Surface(shadow.size, pg.SRCALPHA)
        pg.draw.ellipse(s, (0,0,0,110), s.get_rect())
        self.screen.blit(s, shadow.topleft)
        # body
        body = pg.Rect(0, 0, CAR_L, CAR_W)
        body.center = (cx, cy)
        pg.draw.rect(self.screen, car.color, body, border_radius=8)
        # windshield
        roof = body.inflate(-CAR_L*0.45, -CAR_W*0.25)
        pg.draw.rect(self.screen, (230, 240, 255), roof, border_radius=6)
        # wheels
        for ox in (-CAR_L*0.35, CAR_L*0.35):
            wheel = pg.Rect(0, 0, 8, CAR_W+2)
            wheel.center = (cx + ox, cy)
            pg.draw.rect(self.screen, (20,20,20), wheel, border_radius=3)
        # ambulance blink
        if car.name == "AMB":
            blink = (int(pg.time.get_ticks()/120) % 2) == 0
            if blink:
                pg.draw.rect(self.screen, (255,240,120), (body.right-6, body.top+4, 6, 6), border_radius=2)
                pg.draw.rect(self.screen, (255,240,120), (body.right-6, body.bottom-10, 6, 6), border_radius=2)
        # stroke
        pg.draw.rect(self.screen, (0,0,0), body, 2, border_radius=8)

    def draw_marker(self, wx: float, camera_x: float, color: Tuple[int,int,int], label: str):
        sx = self.world_to_screen(wx, camera_x)
        cy = WIN_H // 2
        pg.draw.line(self.screen, color, (sx, cy - ROAD_W//2 - 16), (sx, cy + ROAD_W//2 + 16), 4)
        tag = self.font.render(label, True, color)
        self.screen.blit(tag, (sx - tag.get_width()//2, cy + ROAD_W//2 + 20))

    def draw_hud(self, world: 'World', camera_x: float, mode: str, paused: bool, show_help: bool, spawn_on: bool):
        lines = [
            f"t={world.t:5.1f}s  EGO x={world.ego.x:6.1f}m  v={world.ego.v:4.1f}  lane={world.ego.lane}",
            f"AMB {'ON ' if world.amb.active else 'OFF'} x={world.amb.x:6.1f} v={world.amb.v:4.1f} lane={world.amb.lane}",
            f"mode=[{mode}]  pause={'YES' if paused else 'no '}  vmax={world.ego.vmax:4.1f}  traffic={'ON' if spawn_on else 'OFF'}",
            f"cars={len(world.traffic)+2}",
        ]
        y = 10
        for s in lines:
            surf = self.font.render(s, True, TEXT)
            self.screen.blit(surf, (12, y))
            y += 20
        if world.accident_x is not None:
            sx = self.world_to_screen(world.accident_x, camera_x)
            pg.draw.circle(self.screen, ACC_COLOR, (int(sx), 40), 7)
            self.screen.blit(self.font.render("ACC", True, ACC_COLOR), (int(sx)-14, 52))
        if world.roadblock_x is not None:
            sx = self.world_to_screen(world.roadblock_x, camera_x)
            pg.draw.circle(self.screen, RB_COLOR, (int(sx), 40), 7)
            self.screen.blit(self.font.render("RB", True, RB_COLOR), (int(sx)-10, 52))
        if show_help:
            self.draw_help_overlay()

    def draw_help_overlay(self):
        pad = 10
        msg = [
            "Controls:",
            "  1: Accident mode (click to place)",
            "  2: Roadblock mode (click to place)",
            "  A / Shift+A: Ambulance now / in 5s",
            "  C: Clear hazard near cursor  |  X: Clear ALL",
            "  R: Responders clear scene (unblocks)",
            "  W/S: Ego speed up/down  |  T: Toggle traffic",
            "  SPACE: Pause/Resume  |  H: Help",
            "  ESC: Back to Menu  |  Q: Quit",
        ]
        w = 560
        h = 22 * len(msg) + pad*2
        rect = pg.Rect(WIN_W - w - 20, 20, w, h)
        s = pg.Surface((rect.w, rect.h), pg.SRCALPHA)
        s.fill((0,0,0,170))
        self.screen.blit(s, rect.topleft)
        y = rect.top + pad
        for line in msg:
            surf = self.font.render(line, True, TEXT)
            self.screen.blit(surf, (rect.left + pad, y))
            y += 22

# ----------------------------- Menu & App -----------------------------
class Menu:
    def __init__(self, screen: pg.Surface, bigfont: pg.font.Font, font: pg.font.Font):
        self.screen = screen
        self.bigfont = bigfont
        self.font = font

    def draw(self):
        self.screen.fill(BG)
        title = self.bigfont.render("Car Simulation", True, TEXT)
        sub = self.font.render("Ambulance · Accident · Roadblock (v2.2)", True, MUTED)
        self.screen.blit(title, (WIN_W//2 - title.get_width()//2, 60))
        self.screen.blit(sub, (WIN_W//2 - sub.get_width()//2, 110))

        btns = [
            ("1) Ambulance preset", 1),
            ("2) Accident preset", 2),
            ("3) Roadblock preset", 3),
            ("Q) Quit", -1),
        ]
        y = 200
        for label, _ in btns:
            surf = self.font.render(label, True, TEXT)
            rect = surf.get_rect(center=(WIN_W//2, y))
            pg.draw.rect(self.screen, (60,60,60), rect.inflate(30,12), border_radius=8)
            self.screen.blit(surf, rect)
            y += 64

        tip = self.font.render("Hazards stop EVERY car at the line. Clear to release.", True, MUTED)
        self.screen.blit(tip, (WIN_W//2 - tip.get_width()//2, WIN_H - 60))

    def interact(self) -> int:
        while True:
            for e in pg.event.get():
                if e.type == pg.QUIT:
                    return -1
                if e.type == pg.KEYDOWN:
                    if e.key == pg.K_1: return 1
                    if e.key == pg.K_2: return 2
                    if e.key == pg.K_3: return 3
                    if e.key in (pg.K_ESCAPE, pg.K_q): return -1
            self.draw()
            pg.display.flip()

class App:
    def __init__(self):
        pg.init()
        pg.display.set_caption("Car Sim v2.2 — Deterministic Stops + Anti-Overlap")
        self.screen = pg.display.set_mode((WIN_W, WIN_H))
        self.font = pg.font.SysFont(FONT_NAME, 16)
        self.bigfont = pg.font.SysFont(FONT_NAME, 42, bold=True)
        self.ui = UI(self.screen, self.font)
        self.clock = pg.time.Clock()

    def run(self):
        while True:
            choice = self.menu()
            if choice == -1:
                break
            self.sim(choice)

    def menu(self) -> int:
        m = Menu(self.screen, self.bigfont, self.font)
        return m.interact()

    def sim(self, choice: int):
        world = World(length=2200.0)
        paused = False
        show_help = True
        mode = "accident"
        camera_x = 0.0

        # presets
        if choice == 1:
            world.schedule_ambulance(8.0)
        elif choice == 2:
            world.place_accident(400.0)
        elif choice == 3:
            world.place_roadblock(900.0)

        running = True
        while running:
            for e in pg.event.get():
                if e.type == pg.QUIT:
                    pg.quit(); raise SystemExit
                if e.type == pg.KEYDOWN:
                    if e.key == pg.K_ESCAPE:
                        running = False
                    elif e.key == pg.K_q:
                        pg.quit(); raise SystemExit
                    elif e.key == pg.K_h:
                        show_help = not show_help
                    elif e.key == pg.K_SPACE:
                        paused = not paused
                    elif e.key == pg.K_1:
                        mode = "accident"
                    elif e.key == pg.K_2:
                        mode = "roadblock"
                    elif e.key == pg.K_a:
                        if pg.key.get_mods() & pg.KMOD_SHIFT:
                            world.schedule_ambulance(5.0)
                        else:
                            world.dispatch_ambulance()
                    elif e.key == pg.K_c:
                        mx, my = pg.mouse.get_pos()
                        wx = camera_x + mx / PX_PER_M
                        nearest = None
                        if world.accident_x is not None:
                            nearest = (abs(world.accident_x - wx), 'acc')
                        if world.roadblock_x is not None:
                            d = abs(world.roadblock_x - wx)
                            if nearest is None or d < nearest[0]:
                                nearest = (d, 'rb')
                        if nearest and nearest[0] < 40:
                            if nearest[1] == 'acc': world.clear_accident()
                            else: world.clear_roadblock()
                    elif e.key == pg.K_x or e.key == pg.K_r:
                        world.clear_accident(); world.clear_roadblock()
                if e.type == pg.MOUSEBUTTONDOWN and e.button == 1:
                    mx, my = e.pos
                    wx = camera_x + mx / PX_PER_M
                    if mode == "accident":
                        world.place_accident(wx)
                    elif mode == "roadblock":
                        world.place_roadblock(wx)
                if e.type == pg.MOUSEBUTTONDOWN and e.button == 3:
                    mx, my = e.pos
                    wx = camera_x + mx / PX_PER_M
                    if world.accident_x is not None and (world.roadblock_x is None or abs(world.accident_x - wx) < abs(world.roadblock_x - wx)):
                        if abs(world.accident_x - wx) < 40:
                            world.clear_accident()
                    elif world.roadblock_x is not None and abs(world.roadblock_x - wx) < 40:
                        world.clear_roadblock()

            if not paused:
                world.step()

            target = world.ego.x - CAM_MARGIN_X
            camera_x += (target - camera_x) * 0.15
            camera_x = max(0.0, camera_x)

            self.screen.fill(BG)
            self.ui.draw_road(camera_x)
            if world.accident_x is not None:
                self.ui.draw_marker(world.accident_x, camera_x, ACC_COLOR, "ACCIDENT")
            if world.roadblock_x is not None:
                self.ui.draw_marker(world.roadblock_x, camera_x, RB_COLOR, "ROADBLOCK")
            if world.amb.active:
                self.ui.draw_car(world.amb, camera_x)
            for c in world.traffic:
                self.ui.draw_car(c, camera_x)
            self.ui.draw_car(world.ego, camera_x)
            self.ui.draw_hud(world, camera_x, mode, paused, show_help, world.spawn_traffic)

            pg.display.flip()
            self.clock.tick(60)

if __name__ == "__main__":
    pg.init()
    App().run()
